In [3]:
import pandas as pd, numpy as np, re, zipfile
from pathlib import Path

excel_path = "Datos finales para análisis IA _proyecto ENTOMOTIVE_.xlsx"

In [10]:
# ---------- helpers ----------
def make_unique_columns(df):
    cols = list(df.columns)
    seen = {}
    new_cols = []
    for c in cols:
        if c not in seen: # First time seen, add as is
            seen[c] = 0
            new_cols.append(c)
        else: # Already exists, add with prefix __dupN
            seen[c] += 1
            new_cols.append(f"{c}__dup{seen[c]}")
    df = df.copy()
    df.columns = new_cols
    return df

def norm_treat(x): # normalize treatment names
    if pd.isna(x):
        return np.nan
    if isinstance(x,(int,np.integer)):
        return str(int(x))
    if isinstance(x,(float,np.floating)):
        if float(x).is_integer():
            return str(int(x))
        return str(x)
    s=str(x).strip()
    if re.fullmatch(r"\d+\.0+", s): # integer float like "3.0"
        return s.split('.')[0]
    return s

def clean_mean_sd_sheet_from_header12(sheet_name):
    df = pd.read_excel(excel_path, sheet_name=sheet_name, header=[1,2])
    first_row = df.iloc[0] # to get subtypes
    new_cols=[]

    for col in df.columns:
        lvl0,lvl1 = col if isinstance(col, tuple) else (col, col)
        base = str(lvl1).strip() # main name
        subtype = str(first_row[col]).strip()
        if base in ["Dieta","Tratamiento"]:
            new_cols.append(base) # This column stays as is
        else:
            if subtype.lower() in ["media","mean"]:
                new_cols.append(f"{base}_media")
            elif subtype.lower() in ["sd","std","desviacion","desviación","desviacion estandar","desviación estándar"]:
                new_cols.append(f"{base}_sd")
            else:
                new_cols.append(base) # No matching subtype, keep base name
    df.columns=new_cols
    df = df.iloc[1:].reset_index(drop=True)
    df = df[df["Dieta"].notna()]
    df = df[df["Dieta"]!="Dieta"]
    df["Dieta"] = df["Dieta"].astype(str).str.strip()
    df["Tratamiento"] = df["Tratamiento"].apply(norm_treat)
    return make_unique_columns(df)

def clean_productivos(sheet_name):
    # Cleaning column names from extra spaces, etc.
    df = pd.read_excel(excel_path, sheet_name=sheet_name, header=2)
    df = df.dropna(subset=["Dieta"]) # remove rows with "Dieta = NaN"
    df = df[df["Dieta"]!="Dieta"].reset_index(drop=True) # remove repeated header rows
    df["Dieta"] = df["Dieta"].astype(str).str.strip()
    df["Tratamiento"] = df["Tratamiento"].apply(norm_treat) # normalize treatment names
    if "FCR " in df.columns and "FCR" not in df.columns:
        df = df.rename(columns={"FCR ":"FCR"})
    return make_unique_columns(df)

def clean_larvas(sheet_name):
    # Cleaning column names from extra spaces, etc.
    df = pd.read_excel(excel_path, sheet_name=sheet_name, header=2)
    df = df.dropna(subset=["Dieta"])
    df = df[df["Dieta"]!="Dieta"].reset_index(drop=True)
    df["Dieta"] = df["Dieta"].astype(str).str.strip()
    df["Tratamiento"] = df["Tratamiento"].apply(norm_treat)
    return make_unique_columns(df)

def clean_tpc(sheet_name):
    df = pd.read_excel(excel_path, sheet_name=sheet_name, header=[1,2])
    df.columns = ["Dieta","Tratamiento","TPC_dieta_media","TPC_dieta_sd","TPC_larva_media","TPC_larva_sd"]
    df = df[df["Dieta"].notna()]
    df = df[df["Dieta"]!="Dieta"].reset_index(drop=True)
    df["Dieta"] = df["Dieta"].astype(str).str.strip()
    df["Tratamiento"] = df["Tratamiento"].apply(norm_treat)
    return make_unique_columns(df)

def add_replica(df, by_cols):
    # Add Replica column based on cumulative count within groups defined by by_cols
    # for example, by_cols = ["Dieta","Tratamiento"] will number replicas for each Dieta-Tratamiento combination
    df = df.copy()
    df["Replica"] = df.groupby(by_cols).cumcount() + 1
    return make_unique_columns(df)

def parse_diet_name(dieta):
    # Parse diet name into structured components for analysis
    s = str(dieta).strip()
    byproduct = None
    inclusion = np.nan
    water = "none"
    processing = "none"
    study_block = "main"

    # In this block, we check for specific diet name patterns and extract relevant info
    '''
    byproduct: type of byproduct used (e.g., cafe, orujillo, control, hoja, orujo, quinoa, other)
    inclusion: percentage of byproduct inclusion in the diet
    water: water condition (dry, agar, humidity, none)
    processing: processing method (lyophilized, oven, none)
    study_block: experimental block (water_control, coffee_orujillo, main)
    '''

    if s.lower().startswith("control ") and any(k in s.lower() for k in ["seco","agar","humedad"]):
        byproduct="control"; inclusion=0; study_block="water_control"
        if "seco" in s.lower(): water="dry"
        elif "agar" in s.lower(): water="agar"
        elif "humedad" in s.lower(): water="humidity"
        return dict(diet_name=s, byproduct_type=byproduct, inclusion_pct=inclusion,
                    water_condition=water, processing=processing, study_block=study_block)
    if s.lower().startswith("posos café"):
        byproduct="cafe"; study_block="coffee_orujillo"
        m=re.search(r"(\d+)", s); inclusion=int(m.group(1)) if m else np.nan
        if "liof" in s.lower(): processing="lyophilized"
        elif "horno" in s.lower(): processing="oven"
        return dict(diet_name=s, byproduct_type=byproduct, inclusion_pct=inclusion,
                    water_condition=water, processing=processing, study_block=study_block)
    if s.lower().startswith("orujillo"):
        byproduct="orujillo"; study_block="coffee_orujillo"
        m=re.search(r"(\d+)", s); inclusion=int(m.group(1)) if m else np.nan
        return dict(diet_name=s, byproduct_type=byproduct, inclusion_pct=inclusion,
                    water_condition=water, processing=processing, study_block=study_block)
    if s.lower()=="control":
        byproduct="control"; inclusion=0
    else:
        m=re.match(r"(Hoja|Orujo|Quinoa)(\d+)", s, flags=re.IGNORECASE)
        if m:
            byproduct=m.group(1).lower(); inclusion=int(m.group(2))
        else:
            byproduct="other"
    return dict(diet_name=s, byproduct_type=byproduct, inclusion_pct=inclusion,
                water_condition=water, processing=processing, study_block=study_block)

def add_ratios(df):
    # Add nutritional ratios to the dataframe
    df=df.copy()
    needed=["Proteína (%)_media","Grasa (%)_media","Fibra (%)_media","Cenizas (%)_media","Carbohidratos (%)_media"]
    if not all(c in df.columns for c in needed):
        return df
    df["ratio_P_C"]=df["Proteína (%)_media"]/df["Carbohidratos (%)_media"]
    df["ratio_P_F"]=df["Proteína (%)_media"]/df["Grasa (%)_media"]
    df["ratio_Fibra_Grasa"]=df["Fibra (%)_media"]/df["Grasa (%)_media"]
    return make_unique_columns(df)

### Testing the functions

Dietas Hermetia

In [12]:
clean_mean_sd_sheet_from_header12("Dietas HERMETIA")

,Dieta,Tratamiento,Proteína (%)_media,Proteína (%)_sd,Grasa (%)_media,Grasa (%)_sd,Fibra (%)_media,Fibra (%)_sd,Cenizas (%)_media,Cenizas (%)_sd,Carbohidratos (%)_media,Carbohidratos (%)_sd
0,Control,0,18.175156,0.435439,2.766018,0.029604,2.513074,0.093525,22.034336,0.09653,54.511416,0.547815
1,Hoja15,1,16.838123,0.258047,2.650274,0.023624,3.702291,0.181163,19.294028,0.081509,57.515283,0.043625
2,Hoja30,2,15.137502,0.23809,2.622729,0.113278,5.223646,0.117353,16.872466,0.190969,60.143657,0.305668
3,Hoja50,3,13.511368,0.191933,2.931321,0.041274,7.773009,0.197489,14.292793,0.163029,61.491509,0.211804
4,Orujo30,4,16.626013,0.017579,5.906098,0.141556,7.152747,0.299425,19.922816,0.156563,50.392325,0.542995
5,Orujo50,5,14.938607,0.268863,7.274675,0.126253,11.503109,0.117809,15.247487,0.318942,51.036123,0.524178
6,Orujo70,6,14.352349,0.212814,9.500662,0.106972,14.993037,0.277609,13.500153,0.313608,47.653799,0.375975
7,Orujo90,7,12.528128,0.461024,10.817841,0.13248,19.978773,0.23035,10.484394,0.378936,46.190864,0.05529
8,Quinoa15,8,16.372397,0.19357,2.716249,0.078263,2.354644,0.029709,19.603305,0.105794,58.953405,0.256057
9,Quinoa30,9,15.457511,0.388899,2.974883,0.104481,2.588872,0.189671,19.048045,0.254833,59.930689,0.82327


Parametros Productivos Hermetia

In [16]:
clean_productivos("Parámetros productivos HERMETIA").head(5)

,Dieta,Tratamiento,FCR,Ganancia de peso (mg) por larva,Ganancia de peso (mg) media diaria por larva,Reducción (%) del sustrato (reducción de la masa de dieta suministrada),Reducción diaria (%) del sustrato (reducción diaria de la masa de dieta suministrada),Peso total de las larvas (g) al finalizar el periodo de alimentación,Peso total del sustrato (g) al finalizar el periodo de alimentación
0,Control,0,1.767204,138.77,11.564167,57.673863,4.806155,1544.0,1884.437753
1,Control,0,1.611845,168.14,14.011667,68.672394,5.722699,1983.5,1391.567280
2,Control,0,1.469797,163.20,13.600000,69.976968,5.831414,2207.5,1334.673548
3,Hoja15,1,1.933048,113.42,9.451667,70.544403,5.878700,1739.5,1330.567333
4,Hoja15,1,1.371372,146.50,12.208333,76.302053,6.358504,2602.5,1069.702232


Hermetia Larvas

In [17]:
clean_larvas("HERMETIA larvas").head(5)

,Dieta,Tratamiento,PROTEINA (%),GRASA (%),QUITINA (%),CENIZAS (%),CARBOHIDRATOS (%)
0,Control,0,32.298228,7.856994,4.536508,24.085233,31.223037
1,Control,0,31.988273,9.178856,3.188548,23.805322,31.839000
2,Control,0,31.522017,17.458388,4.024695,22.767007,24.227893
3,Hoja15,1,32.439877,8.138716,10.804604,25.394843,23.221962
4,Hoja15,1,31.861333,9.794866,7.392563,26.810799,24.140439


TPC Hermetia

In [19]:
clean_tpc("TPC HERMETIA").head(5)

,Dieta,Tratamiento,TPC_dieta_media,TPC_dieta_sd,TPC_larva_media,TPC_larva_sd
0,Control,0,0.180416,0.012046,0.432843,0.031135
1,Hoja15,1,0.90341,0.100583,0.358376,0.017351
2,Hoja30,2,1.616542,0.050153,0.442705,0.046997
3,Hoja50,3,2.484336,0.3436,0.31179,0.020487
4,Orujo30,4,0.785741,0.110158,0.351143,0.044412


In [ ]:
prod_T = clean_productivos("Parámetro productivos TENEBRIO")
prod_T_l = add_replica(prod_T, ["Dieta"])
larv_T_l = add_replica(larv_T, ["Dieta"])
prod_T_l["species"]="Tenebrio"; larv_T_l["species"]="Tenebrio"
# prod_T_l = add_comp_keys_T(prod_T_l)
# larv_T_l = add_comp_keys_T(larv_T_l)

prod_T_l.head(5)


,Dieta,Tratamiento,FCR,Ganancia de peso (mg) por larva,Ganancia de peso (mg) media diaria por larva,Reducción (%) del sustrato (reducción de la masa de dieta suministrada),Reducción diaria (%) del sustrato (reducción diaria de la masa de dieta suministrada),Peso total de las larvas (g) al finalizar el periodo de alimentación,Peso total del sustrato (g) al finalizar el periodo de alimentación,Replica,species,Dieta_comp,Trat_comp
0,Control,0,2.965587,NaN,NaN,34.183333,4.883333,836,3949,1,Tenebrio,Control,0
1,Control,0,2.773211,41.5,0.846939,30.083333,4.297619,723,4195,2,Tenebrio,Control,0
2,Control,0,2.727205,48,0.979592,30.533333,4.361905,745,4168,3,Tenebrio,Control,0
3,Hoja15,1,2.542555,37.1,0.757143,21.633333,3.090476,613,4702,1,Tenebrio,Hoja15,1
4,Hoja15,1,2.855694,37.8,0.771429,20.65,2.95,506,4761,2,Tenebrio,Hoja15,1


In [ ]:

# Larva dataframe joined with productivity dataframe
prod_T_ds = prod_T_l.merge(larv_T_l, on=["Dieta","Replica","species"], suffixes=("_prod","_larv"))

prod_T_ds["Dieta_comp"]=prod_T_ds["Dieta_comp_prod"]
prod_T_ds["Trat_comp"]=prod_T_ds["Trat_comp_prod"]
prod_T_ds["Tratamiento"]=prod_T_ds["Tratamiento_prod"]

prod_T_ds.head(5)


,Dieta,Tratamiento_prod,FCR,Ganancia de peso (mg) por larva,Ganancia de peso (mg) media diaria por larva,Reducción (%) del sustrato (reducción de la masa de dieta suministrada),Reducción diaria (%) del sustrato (reducción diaria de la masa de dieta suministrada),Peso total de las larvas (g) al finalizar el periodo de alimentación,Peso total del sustrato (g) al finalizar el periodo de alimentación,Replica,...,Dieta_comp_prod,Trat_comp_prod,Tratamiento_larv,PROTEINA (%),GRASA (%),QUITINA (%),CENIZAS (%),CARBOHIDRATOS (%),Dieta_comp_larv,Trat_comp_larv
0,Control,0,2.965587,NaN,NaN,34.183333,4.883333,836,3949,1,...,Control,0,0,42.724278,37.218749,5.87989,3.733378,10.443705,Control,0
1,Control,0,2.773211,41.5,0.846939,30.083333,4.297619,723,4195,2,...,Control,0,0,43.798393,32.105928,6.668637,4.028978,13.398064,Control,0
2,Control,0,2.727205,48,0.979592,30.533333,4.361905,745,4168,3,...,Control,0,0,43.331882,36.23092,6.428088,4.041571,9.967538,Control,0
3,Hoja15,1,2.542555,37.1,0.757143,21.633333,3.090476,613,4702,1,...,Hoja15,1,1,44.599669,35.773419,5.862071,4.110244,9.654598,Hoja15,1
4,Hoja15,1,2.855694,37.8,0.771429,20.65,2.95,506,4761,2,...,Hoja15,1,1,48.582073,33.395068,6.009969,4.40894,7.60395,Hoja15,1


### Creating the Dataset

#### Loading data

In [43]:

diet_H = add_ratios(clean_mean_sd_sheet_from_header12("Dietas HERMETIA"))
diet_T = add_ratios(clean_mean_sd_sheet_from_header12("Dietas TENEBRIO"))
prod_H = clean_productivos("Parámetros productivos HERMETIA")
prod_T = clean_productivos("Parámetro productivos TENEBRIO")
larv_H = clean_larvas("HERMETIA larvas")
larv_T = clean_larvas("TENEBRIO larvas")
tpc_H = clean_tpc("TPC HERMETIA")
tpc_T = clean_tpc("TPC TENEBRIO")

control_variants = {"Control Seco","Control Agar","Control Humedad superf"}
def add_comp_keys_T(df):
    df=df.copy()
    df["Dieta_comp"]=df["Dieta"]
    df["Trat_comp"]=df["Tratamiento"]
    mask = df["Dieta"].isin(control_variants) | df["Tratamiento"].isin(["A.1","A.2","A.3"])
    df.loc[mask,"Dieta_comp"]="Control"
    df.loc[mask,"Trat_comp"]="A"
    return make_unique_columns(df)


#### Productivity Dataset

In [44]:

# Hermetia
prod_H_l = add_replica(prod_H, ["Dieta","Tratamiento"]) # Enumerate lots/replicas
larv_H_l = add_replica(larv_H, ["Dieta","Tratamiento"])
prod_H_l["species"]="Hermetia"; larv_H_l["species"]="Hermetia"
prod_H_ds = prod_H_l.merge(larv_H_l, on=["Dieta","Tratamiento","Replica","species"], suffixes=("_prod","_larv"))
prod_H_ds = prod_H_ds.merge(diet_H, on=["Dieta","Tratamiento"], how="left")
prod_H_ds = prod_H_ds.merge(tpc_H, on=["Dieta","Tratamiento"], how="left")
meta_H = pd.DataFrame([parse_diet_name(d) for d in prod_H_ds["Dieta"]])
prod_H_ds = make_unique_columns(pd.concat([prod_H_ds.reset_index(drop=True), meta_H.reset_index(drop=True)], axis=1))

# Tenebrio
prod_T_l = add_replica(prod_T, ["Dieta"])
larv_T_l = add_replica(larv_T, ["Dieta"])
prod_T_l["species"]="Tenebrio"; larv_T_l["species"]="Tenebrio"
prod_T_l = add_comp_keys_T(prod_T_l)
larv_T_l = add_comp_keys_T(larv_T_l)

# Joining Larva dataframe with productivity dataframe
prod_T_ds = prod_T_l.merge(larv_T_l, on=["Dieta","Replica","species"], suffixes=("_prod","_larv"))
prod_T_ds["Dieta_comp"]=prod_T_ds["Dieta_comp_prod"]
prod_T_ds["Trat_comp"]=prod_T_ds["Trat_comp_prod"]
prod_T_ds["Tratamiento"]=prod_T_ds["Tratamiento_prod"]
prod_T_ds = prod_T_ds.merge(diet_T, left_on=["Dieta_comp","Trat_comp"], right_on=["Dieta","Tratamiento"], how="left", suffixes=("","_diet"))

# Remove redundant columns after merge
for k in ["Dieta_diet","Tratamiento_diet"]:
    if k in prod_T_ds.columns: prod_T_ds = prod_T_ds.drop(columns=[k])

# Merge TPC data to productivity dataframe
prod_T_ds = prod_T_ds.merge(tpc_T, left_on=["Dieta_comp","Trat_comp"], right_on=["Dieta","Tratamiento"], how="left", suffixes=("","_tpc"))
for k in ["Dieta_tpc","Tratamiento_tpc"]:
    if k in prod_T_ds.columns: prod_T_ds = prod_T_ds.drop(columns=[k])

# Add parsed diet metadata
meta_T = pd.DataFrame([parse_diet_name(d) for d in prod_T_ds["Dieta"]])
prod_T_ds = make_unique_columns(pd.concat([prod_T_ds.reset_index(drop=True), meta_T.reset_index(drop=True)], axis=1))

prod_T_ds.head(5)

,Dieta,Tratamiento_prod,FCR,Ganancia de peso (mg) por larva,Ganancia de peso (mg) media diaria por larva,Reducción (%) del sustrato (reducción de la masa de dieta suministrada),Reducción diaria (%) del sustrato (reducción diaria de la masa de dieta suministrada),Peso total de las larvas (g) al finalizar el periodo de alimentación,Peso total del sustrato (g) al finalizar el periodo de alimentación,Replica,...,TPC_dieta_media,TPC_dieta_sd,TPC_larva_media,TPC_larva_sd,diet_name,byproduct_type,inclusion_pct,water_condition,processing,study_block
0,Control,0,2.965587,NaN,NaN,34.183333,4.883333,836,3949,1,...,0.231935,0.002182,0.321229,0.023088,Control,control,0,none,none,main
1,Control,0,2.773211,41.5,0.846939,30.083333,4.297619,723,4195,2,...,0.231935,0.002182,0.321229,0.023088,Control,control,0,none,none,main
2,Control,0,2.727205,48,0.979592,30.533333,4.361905,745,4168,3,...,0.231935,0.002182,0.321229,0.023088,Control,control,0,none,none,main
3,Hoja15,1,2.542555,37.1,0.757143,21.633333,3.090476,613,4702,1,...,0.993719,0.052006,0.363926,0.029128,Hoja15,hoja,15,none,none,main
4,Hoja15,1,2.855694,37.8,0.771429,20.65,2.95,506,4761,2,...,0.993719,0.052006,0.363926,0.029128,Hoja15,hoja,15,none,none,main


In [45]:
def reorder_productivity(df):
    id_cols = ["species","study_block","diet_name","Dieta","Tratamiento","Replica","byproduct_type","inclusion_pct","water_condition","processing"]
    id_cols=[c for c in id_cols if c in df.columns]
    diet_cols=[c for c in df.columns if c.endswith("_media") or c.endswith("_sd") or c.startswith("ratio_")]
    y_prod=[
        "FCR",
        "Ganancia de peso (mg) por larva",
        "Ganancia de peso (mg) media diaria por larva",
        "Reducción (%) del sustrato (reducción de la masa de dieta suministrada)",
        "Reducción diaria (%) del sustrato (reducción diaria de la masa de dieta suministrada)",
        "Peso total de las larvas (g) al finalizar el periodo de alimentación",
        "Peso total del sustrato (g) al finalizar el periodo de alimentación",
    ]
    y_prod=[c for c in y_prod if c in df.columns]
    larv_cols=[c for c in ["PROTEINA (%)","GRASA (%)","QUITINA (%)","CENIZAS (%)","CARBOHIDRATOS (%)"] if c in df.columns]
    tpc_cols=[c for c in ["TPC_dieta_media","TPC_dieta_sd","TPC_larva_media","TPC_larva_sd"] if c in df.columns]
    remaining=[c for c in df.columns if c not in set(id_cols+diet_cols+y_prod+larv_cols+tpc_cols)]
    return make_unique_columns(df[id_cols+diet_cols+y_prod+larv_cols+tpc_cols+remaining])

In [46]:
prod_H_ds = reorder_productivity(prod_H_ds)
prod_T_ds = reorder_productivity(prod_T_ds)
prod_all_ds = pd.concat([make_unique_columns(prod_H_ds), make_unique_columns(prod_T_ds)], ignore_index=True, sort=False)

prod_all_ds

,species,study_block,diet_name,Dieta,Tratamiento,Replica,byproduct_type,inclusion_pct,water_condition,processing,...,TPC_larva_media__dup1,TPC_larva_sd__dup1,Tratamiento_prod,Dieta_comp_prod,Trat_comp_prod,Tratamiento_larv,Dieta_comp_larv,Trat_comp_larv,Dieta_comp,Trat_comp
0,Hermetia,main,Control,Control,0,1,control,0,none,none,...,0.432843,0.031135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Hermetia,main,Control,Control,0,2,control,0,none,none,...,0.432843,0.031135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Hermetia,main,Control,Control,0,3,control,0,none,none,...,0.432843,0.031135,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Hermetia,main,Hoja15,Hoja15,1,1,hoja,15,none,none,...,0.358376,0.017351,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Hermetia,main,Hoja15,Hoja15,1,2,hoja,15,none,none,...,0.358376,0.017351,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85,Tenebrio,coffee_orujillo,Orujillo30,Orujillo30,E,2,orujillo,30,none,none,...,NaN,NaN,E,Orujillo30,E,NaN,Orujillo30,NaN,Orujillo30,E
86,Tenebrio,coffee_orujillo,Orujillo30,Orujillo30,E,3,orujillo,30,none,none,...,NaN,NaN,E,Orujillo30,E,NaN,Orujillo30,NaN,Orujillo30,E
87,Tenebrio,coffee_orujillo,Orujillo50,Orujillo50,F,1,orujillo,50,none,none,...,NaN,NaN,F,Orujillo50,F,NaN,Orujillo50,NaN,Orujillo50,F
88,Tenebrio,coffee_orujillo,Orujillo50,Orujillo50,F,2,orujillo,50,none,none,...,NaN,NaN,F,Orujillo50,F,NaN,Orujillo50,NaN,Orujillo50,F


#### Quality Dataset

In [73]:
def larva_agg_by_diet(df):
    cols=[c for c in ["PROTEINA (%)","GRASA (%)","QUITINA (%)","CENIZAS (%)","CARBOHIDRATOS (%)"] if c in df.columns]
    g=df.groupby("Dieta")[cols].agg(["mean","std"]).reset_index()
    new_cols=["Dieta"]
    for c in cols:
        new_cols += [f"{c}_mean", f"{c}_sd"]
    g.columns=new_cols
    return make_unique_columns(g)

Hermetia composition

In [74]:
qual_H = diet_H.copy()
qual_H["species"]="Hermetia"

# Add TPC and larva aggregated data
qual_H = qual_H.merge(tpc_H, on=["Dieta","Tratamiento"], how="left")

# Larva aggregrated data refers to all replicas of each diet with same diet and treatment
qual_H = qual_H.merge(larva_agg_by_diet(larv_H), on="Dieta", how="left")

qual_H = make_unique_columns(pd.concat([qual_H.reset_index(drop=True),
                                        pd.DataFrame([parse_diet_name(d) for d in qual_H["Dieta"]]).reset_index(drop=True)], axis=1))
qual_H.head(5)

,Dieta,Tratamiento,Proteína (%)_media,Proteína (%)_sd,Grasa (%)_media,Grasa (%)_sd,Fibra (%)_media,Fibra (%)_sd,Cenizas (%)_media,Cenizas (%)_sd,...,CENIZAS (%)_mean,CENIZAS (%)_sd,CARBOHIDRATOS (%)_mean,CARBOHIDRATOS (%)_sd,diet_name,byproduct_type,inclusion_pct,water_condition,processing,study_block
0,Control,0,18.175156,0.435439,2.766018,0.029604,2.513074,0.093525,22.034336,0.09653,...,23.552521,0.694522,29.096643,4.227694,Control,control,0,none,none,main
1,Hoja15,1,16.838123,0.258047,2.650274,0.023624,3.702291,0.181163,19.294028,0.081509,...,26.271347,0.765791,24.788409,1.971962,Hoja15,hoja,15,none,none,main
2,Hoja30,2,15.137502,0.23809,2.622729,0.113278,5.223646,0.117353,16.872466,0.190969,...,20.199422,1.128735,24.414996,3.130644,Hoja30,hoja,30,none,none,main
3,Hoja50,3,13.511368,0.191933,2.931321,0.041274,7.773009,0.197489,14.292793,0.163029,...,20.929722,0.207798,29.368771,2.589442,Hoja50,hoja,50,none,none,main
4,Orujo30,4,16.626013,0.017579,5.906098,0.141556,7.152747,0.299425,19.922816,0.156563,...,20.538848,1.106628,30.074157,5.733730,Orujo30,orujo,30,none,none,main


Tenebrio Composition

In [75]:
qual_T = pd.DataFrame({"Dieta": sorted(larv_T["Dieta"].unique())})
qual_T["species"]="Tenebrio"
treat_lookup = prod_T.groupby("Dieta")["Tratamiento"].apply(lambda s: next((x for x in s if pd.notna(x)), np.nan)).to_dict()
treat_lookup

{'Control': '0',
 'Control Agar': 'A.2',
 'Control Humedad superf': 'A.3',
 'Control Seco': 'A.1',
 'Hoja15': '1',
 'Hoja30': '2',
 'Hoja50': '3',
 'Orujillo15': 'D',
 'Orujillo30': 'E',
 'Orujillo50': 'F',
 'Orujo30': '4',
 'Orujo50': '5',
 'Orujo70': '6',
 'Orujo90': '7',
 'Posos café horno15': 'C',
 'Posos café liof15': 'B',
 'Quinoa15': '8',
 'Quinoa30': '9',
 'Quinoa50': '10'}

In [76]:
def comp_pair(dieta):
    if dieta in control_variants:
        return ("Control","A")
    tr=treat_lookup.get(dieta, np.nan)
    if tr in ["A.1","A.2","A.3"]:
        return ("Control","A")
    return (dieta, tr)

pairs=[comp_pair(d) for d in qual_T["Dieta"]]
qual_T["Dieta_comp"]=[p[0] for p in pairs]
qual_T["Trat_comp"]=[p[1] for p in pairs]

qual_T

,Dieta,species,Dieta_comp,Trat_comp
0,Control,Tenebrio,Control,0
1,Control Agar,Tenebrio,Control,A
2,Control Humedad superf,Tenebrio,Control,A
3,Control Seco,Tenebrio,Control,A
4,Hoja15,Tenebrio,Hoja15,1
5,Hoja30,Tenebrio,Hoja30,2
6,Hoja50,Tenebrio,Hoja50,3
7,Orujillo15,Tenebrio,Orujillo15,D
8,Orujillo30,Tenebrio,Orujillo30,E
9,Orujillo50,Tenebrio,Orujillo50,F


In [77]:

# Join diet data to quality dataframe with composition keys
qual_T = qual_T.merge(diet_T, left_on=["Dieta_comp","Trat_comp"], right_on=["Dieta","Tratamiento"], how="left", suffixes=("","_diet"))
for k in ["Dieta_diet","Tratamiento_diet"]:
    if k in qual_T.columns: qual_T = qual_T.drop(columns=[k])

# Merge TPC data to quality dataframe
qual_T = qual_T.merge(tpc_T, left_on=["Dieta_comp","Trat_comp"], right_on=["Dieta","Tratamiento"], how="left", suffixes=("","_tpc"))
for k in ["Dieta_tpc","Tratamiento_tpc"]:
    if k in qual_T.columns: qual_T = qual_T.drop(columns=[k])

# Merge larva aggregated data to quality dataframe
qual_T = qual_T.merge(larva_agg_by_diet(larv_T), on="Dieta", how="left")
qual_T = make_unique_columns(pd.concat([qual_T.reset_index(drop=True),
                                        pd.DataFrame([parse_diet_name(d) for d in qual_T["Dieta"]]).reset_index(drop=True)], axis=1))
qual_T.head(5)

,Dieta,species,Dieta_comp,Trat_comp,Tratamiento,Proteína (%)_media,Proteína (%)_sd,Grasa (%)_media,Grasa (%)_sd,Fibra (%)_media,...,CENIZAS (%)_mean,CENIZAS (%)_sd,CARBOHIDRATOS (%)_mean,CARBOHIDRATOS (%)_sd,diet_name,byproduct_type,inclusion_pct,water_condition,processing,study_block
0,Control,Tenebrio,Control,0,0,14.567601,0.33907,2.847048,0.065187,2.415439,...,3.934642,0.174414,11.269769,1.858471,Control,control,0,none,none,main
1,Control Agar,Tenebrio,Control,A,A,12.755333,0.548518,1.246667,0.073711,NaN,...,3.432133,0.040046,NaN,NaN,Control Agar,control,0,agar,none,water_control
2,Control Humedad superf,Tenebrio,Control,A,A,12.755333,0.548518,1.246667,0.073711,NaN,...,3.378,0.034875,NaN,NaN,Control Humedad superf,control,0,humidity,none,water_control
3,Control Seco,Tenebrio,Control,A,A,12.755333,0.548518,1.246667,0.073711,NaN,...,3.592833,0.028178,NaN,NaN,Control Seco,control,0,dry,none,water_control
4,Hoja15,Tenebrio,Hoja15,1,1,12.9116,0.186512,2.477307,0.104573,3.813707,...,4.134092,0.263735,9.751356,2.197384,Hoja15,hoja,15,none,none,main


In [ ]:
def reorder_quality(df):
    # Reorder columns in quality dataframe creating a consistent structure based on id, diet, TPC, and quality columns
    id_cols=["species","study_block","diet_name","Dieta","byproduct_type","inclusion_pct","water_condition","processing"]

    # Check if id cols exist in dataframe
    id_cols=[c for c in id_cols if c in df.columns]

    # Identify diet, tpc, quality and other related columns
    diet_cols=[c for c in df.columns if c.endswith("_media") or c.endswith("_sd") or c.startswith("ratio_")]
    tpc_diet=[c for c in ["TPC_dieta_media","TPC_dieta_sd"] if c in df.columns]
    y_quality=[c for c in df.columns if c.endswith("_mean")] + [c for c in ["TPC_larva_media","TPC_larva_sd"] if c in df.columns]
    remaining=[c for c in df.columns if c not in set(id_cols+diet_cols+tpc_diet+y_quality)]

    return make_unique_columns(df[id_cols+diet_cols+tpc_diet+y_quality+remaining])



In [79]:
qual_H = reorder_quality(qual_H)
qual_H.head(5)

,species,study_block,diet_name,Dieta,byproduct_type,inclusion_pct,water_condition,processing,Proteína (%)_media,Proteína (%)_sd,...,TPC_dieta_media__dup1,TPC_dieta_sd__dup1,PROTEINA (%)_mean,GRASA (%)_mean,QUITINA (%)_mean,CENIZAS (%)_mean,CARBOHIDRATOS (%)_mean,TPC_larva_media__dup1,TPC_larva_sd__dup1,Tratamiento
0,Hermetia,main,Control,Control,control,0,none,none,18.175156,0.435439,...,0.180416,0.012046,31.936173,11.498079,3.916584,23.552521,29.096643,0.432843,0.031135,0
1,Hermetia,main,Hoja15,Hoja15,hoja,15,none,none,16.838123,0.258047,...,0.90341,0.100583,31.417118,8.225027,9.298099,26.271347,24.788409,0.358376,0.017351,1
2,Hermetia,main,Hoja30,Hoja30,hoja,30,none,none,15.137502,0.23809,...,1.616542,0.050153,24.417325,21.223611,9.744646,20.199422,24.414996,0.442705,0.046997,2
3,Hermetia,main,Hoja50,Hoja50,hoja,50,none,none,13.511368,0.191933,...,2.484336,0.3436,23.370496,14.909885,11.421126,20.929722,29.368771,0.31179,0.020487,3
4,Hermetia,main,Orujo30,Orujo30,orujo,30,none,none,16.626013,0.017579,...,0.785741,0.110158,30.321931,10.156761,8.908303,20.538848,30.074157,0.351143,0.044412,4


In [80]:
qual_T = reorder_quality(qual_T)
qual_T.head(5)

,species,study_block,diet_name,Dieta,byproduct_type,inclusion_pct,water_condition,processing,Proteína (%)_media,Proteína (%)_sd,...,PROTEINA (%)_mean,GRASA (%)_mean,QUITINA (%)_mean,CENIZAS (%)_mean,CARBOHIDRATOS (%)_mean,TPC_larva_media__dup1,TPC_larva_sd__dup1,Dieta_comp,Trat_comp,Tratamiento
0,Tenebrio,main,Control,Control,control,0,none,none,14.567601,0.33907,...,43.284851,35.185199,6.325538,3.934642,11.269769,0.321229,0.023088,Control,0,0
1,Tenebrio,water_control,Control Agar,Control Agar,control,0,agar,none,12.755333,0.548518,...,36.268667,36.186667,NaN,3.432133,NaN,NaN,NaN,Control,A,A
2,Tenebrio,water_control,Control Humedad superf,Control Humedad superf,control,0,humidity,none,12.755333,0.548518,...,39.188333,37.466667,NaN,3.378,NaN,NaN,NaN,Control,A,A
3,Tenebrio,water_control,Control Seco,Control Seco,control,0,dry,none,12.755333,0.548518,...,42.037667,31.69,NaN,3.592833,NaN,NaN,NaN,Control,A,A
4,Tenebrio,main,Hoja15,Hoja15,hoja,15,none,none,12.9116,0.186512,...,45.721612,34.556509,5.836431,4.134092,9.751356,0.363926,0.029128,Hoja15,1,1


Full Quality Dataset

In [81]:
qual_all = pd.concat([make_unique_columns(qual_H), make_unique_columns(qual_T)], ignore_index=True, sort=False)
qual_all.head(5)

,species,study_block,diet_name,Dieta,byproduct_type,inclusion_pct,water_condition,processing,Proteína (%)_media,Proteína (%)_sd,...,PROTEINA (%)_mean,GRASA (%)_mean,QUITINA (%)_mean,CENIZAS (%)_mean,CARBOHIDRATOS (%)_mean,TPC_larva_media__dup1,TPC_larva_sd__dup1,Tratamiento,Dieta_comp,Trat_comp
0,Hermetia,main,Control,Control,control,0,none,none,18.175156,0.435439,...,31.936173,11.498079,3.916584,23.552521,29.096643,0.432843,0.031135,0,NaN,NaN
1,Hermetia,main,Hoja15,Hoja15,hoja,15,none,none,16.838123,0.258047,...,31.417118,8.225027,9.298099,26.271347,24.788409,0.358376,0.017351,1,NaN,NaN
2,Hermetia,main,Hoja30,Hoja30,hoja,30,none,none,15.137502,0.23809,...,24.417325,21.223611,9.744646,20.199422,24.414996,0.442705,0.046997,2,NaN,NaN
3,Hermetia,main,Hoja50,Hoja50,hoja,50,none,none,13.511368,0.191933,...,23.370496,14.909885,11.421126,20.929722,29.368771,0.31179,0.020487,3,NaN,NaN
4,Hermetia,main,Orujo30,Orujo30,orujo,30,none,none,16.626013,0.017579,...,30.321931,10.156761,8.908303,20.538848,30.074157,0.351143,0.044412,4,NaN,NaN


### Save in CSV + Zip

In [ ]:
out_dir = Path("C:\\Users\\maria\\Downloads\\Jose_lenovo\\TFG\\surrogate_models\\data\\entomotive_datasets")
out_dir.mkdir(parents=True, exist_ok=True)

csv_files=[]
def save_csv(df, name):
    p = out_dir / f"{name}.csv"
    df.to_csv(p, index=False)
    csv_files.append(p)

save_csv(prod_H_ds, "productivity_hermetia_lote")
save_csv(prod_T_ds, "productivity_tenebrio_lote")
save_csv(prod_all_ds, "productivity_all_lote")

save_csv(qual_H, "quality_hermetia_dieta")
save_csv(qual_T, "quality_tenebrio_dieta")
save_csv(qual_all, "quality_all_dieta")

zip_path = out_dir / "entomotive_datasets.zip"
with zipfile.ZipFile(zip_path, "w", compression=zipfile.ZIP_DEFLATED) as z:
    for f in csv_files:
        z.write(f, arcname=f.name)

### Preview

In [ ]:

print("Productividad (Hermetia) - fila=lote", prod_H_ds.head(8))
print("Productividad (Tenebrio) - fila=lote", prod_T_ds.head(8))
print("Calidad (Hermetia) - fila=dieta", qual_H.head(8))
print("Calidad (Tenebrio) - fila=dieta", qual_T.head(8))

sizes = pd.DataFrame([
    ["productivity_hermetia_lote", *prod_H_ds.shape],
    ["productivity_tenebrio_lote", *prod_T_ds.shape],
    ["productivity_all_lote", *prod_all_ds.shape],
    ["quality_hermetia_dieta", *qual_H.shape],
    ["quality_tenebrio_dieta", *qual_T.shape],
    ["quality_all_dieta", *qual_all.shape],
], columns=["dataset","rows","cols"])
print("Resumen tamaños datasets", sizes)

zip_path


Productividad (Hermetia) - fila=lote     species study_block diet_name    Dieta Tratamiento  Replica  \
0  Hermetia        main   Control  Control           0        1   
1  Hermetia        main   Control  Control           0        2   
2  Hermetia        main   Control  Control           0        3   
3  Hermetia        main    Hoja15   Hoja15           1        1   
4  Hermetia        main    Hoja15   Hoja15           1        2   
5  Hermetia        main    Hoja15   Hoja15           1        3   
6  Hermetia        main    Hoja30   Hoja30           2        1   
7  Hermetia        main    Hoja30   Hoja30           2        2   

  byproduct_type  inclusion_pct water_condition processing  ...  \
0        control              0            none       none  ...   
1        control              0            none       none  ...   
2        control              0            none       none  ...   
3           hoja             15            none       none  ...   
4           hoja        

WindowsPath('C:/Users/maria/Downloads/Jose_lenovo/TFG/surrogate_models/data/entomotive_datasets/entomotive_datasets.zip')